In [1]:
import json
from pymatgen.core.structure import Structure as ST

In [2]:
import sys

sys.path.append("../")

### Feature generation

This step we are generating features from the crystal structure for the offspring machine learning training. For the prediction of solid-state NMR parameters, the information correlated with the local environment (atomic positions, density, species, etc.) is very important since NMR spectra are heavy influenced by the local electronic density distribution. 

Thus, here we are proposing two set of features for the predciton of the quadrupolar coupling constant of <sup>27</sup>Al species. 

1. A set of pure structure based feature, which denote simple local geometry information such as statistics of bond lengths, bond angles and geometrical distortions (totally 9 features). First order bond lengths and first order bond angles are denoted as 'fbl' and 'fba' here.

2. Three sets of features encoding the intrinsic elemental information denoted as number indexs. The details of implementation can be sound in the source code. 

Read and reload the filtered data from data/interim.

In [4]:
path = "../data/interim/filtered_data.json"
with open(path, "r") as file:
    data_reload = json.load(file)
for data in data_reload:
    data["structure"] = ST.from_dict(data["structure"])
print("number of structures:", len(data_reload))

number of structures: 1681


Calculate the features and store the result together with corresponding NMR params (labels) in a Dataframe

In [5]:
from src.features_gen import concat_features_and_nmr

nmr_struc_data = concat_features_and_nmr(data_reload)
nmr_struc_data.reset_index(drop=True, inplace=True)
nmr_struc_data.head()

  0%|          | 0/1681 [00:00<?, ?it/s]/mnt/c/Users/wusha/Documents/GitHub/27Al_CQ_prediction/notebooks/../src/features_gen.py:591: RuntimeWarning: divide by zero encountered in reciprocal
  dis_matrix_rev = np.reciprocal(distance_matrix(coords_all, coords_all))
  0%|          | 2/1681 [00:00<04:01,  6.95it/s]/home/he/anaconda3/envs/27al_ml/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:3938: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
100%|██████████| 1681/1681 [12:11<00:00,  2.30it/s]


There are 0 structures returns error. Their index are []
error_messages:
 []


,is_O,atom_combination,composition,max_ce,diso,etaQ,CQ,fbl_average,fbl_std,fbl_max,...,110,111,112,113,114,115,116,117,118,119
0,True,O,"[O, O, O, O, O, O]",O:6,202.417197,0.001339,0.264794,1.914520,0.083396,1.990656,...,7.808661e+06,9343.650957,6054.103924,5832.344530,5832.341162,533.498566,145.040097,132.941405,132.941328,95.242235
1,True,O,"[O, O, O, O, O, O]",O:6,202.415778,0.000000,0.264794,1.914520,0.083396,1.990656,...,7.808661e+06,9343.650957,6054.103924,5832.344530,5832.341162,533.498566,145.040097,132.941405,132.941328,95.242235
2,True,O,"[O, O, O, O, O, O]",O:6,202.418564,0.000000,0.264794,1.914520,0.083396,1.990656,...,7.808661e+06,9343.650957,6054.103924,5832.344530,5832.341162,533.498566,145.040097,132.941405,132.941328,95.242235
3,True,O,"[O, O, O, O, O, O]",O:6,202.418705,0.001339,0.264794,1.914520,0.083396,1.990656,...,7.808661e+06,9343.650957,6054.103924,5832.344530,5832.341162,533.498566,145.040097,132.941405,132.941328,95.242235
4,True,O,"[O, O, O, O, O, O]",O:6,196.472860,0.670647,1.521078,1.918155,0.032340,1.959752,...,7.808556e+06,8985.063376,6145.456281,5999.586723,5887.848974,529.391058,140.079201,136.754511,134.578866,97.192860


Save the output result in data/processed for model training.

In [6]:
nmr_struc_data.to_csv("../data/processed/nmr_param_and_features.csv", index=False)